In [544]:
import pandas as pd

In [545]:
import numpy as np

In [546]:
frame=pd.read_csv("titanic.csv")

In [547]:
frame.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [548]:
frame.drop(columns=["PassengerId","Name","Ticket","Cabin","Embarked"],inplace=True)

In [549]:
frame.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [550]:
frame["flag_sex"]=0
frame.loc[frame["Sex"]=="male","flag_sex"]=1

In [551]:
frame.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
flag_sex      0
dtype: int64

In [552]:
frame["Age"].fillna(frame["Age"].median(),inplace=True)

In [553]:
frame.drop(columns=["Sex"],inplace=True)

In [554]:
frame.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,flag_sex
0,0,3,22.0,1,0,7.2500,1
1,1,1,38.0,1,0,71.2833,0
2,1,3,26.0,0,0,7.9250,0
3,1,1,35.0,1,0,53.1000,0
4,0,3,35.0,0,0,8.0500,1


In [555]:
frame.index

RangeIndex(start=0, stop=891, step=1)

In [556]:
frame["Age"].iloc[9]

14.0

In [557]:
def divide_data(x_train,fkey,fval):
    left=pd.DataFrame([],columns=x_train.columns)
    right=pd.DataFrame([],columns=x_train.columns)
    for i in range(len(x_train)):
        if x_train[fkey].iloc[i]>fval:
            right=right.append(x_train.iloc[i])
        else:
            left=left.append(x_train.iloc[i])
    

    return left,right

In [558]:
def entropy(data):
    
    p=(data["Survived"].sum())/len(data)    
    return(-p*np.log2(p)-(1-p)*np.log2(1-p))

In [559]:
def info_val(x_train,fkey,fval):
    left,right=divide_data(x_train,fkey,fval)
    if right.shape[0]==0 or left.shape[0]==0:
        return -10000    
    gain=entropy(x_train)-(len(left)/len(x_train))*entropy(left)-(len(right)/len(x_train))*entropy(right)   
    return gain

In [631]:
class decision_tree():
    def __init__(self,max_depth=8,depth=0):
        self.fkey=None
        self.left=None
        self.right=None
        self.fval=None
        self.max_depth=max_depth
        self.depth=depth
        self.target=None
        
    def train(self,x_train):
        col=list(x_train.drop(columns=["Survived"]).columns)
        info_gain=[]
        for fkey in (col):
            fval=x_train[fkey].mean()
            info_gain.append(info_val(x_train,fkey,fval))
        self.fkey=col[np.argmax(np.array(info_gain))]
        self.fval=x_train[self.fkey].mean()
        print(self.fkey)
        ##Truly left or right node:
        left,right=divide_data(x_train,self.fkey,self.fval)
        if ((left.shape[0]==0)|(right.shape[0]==0)):
            if(x_train["Survived"].mean()>=0.5):
                self.target=1
            else:
                self.target=0
            return
        if(self.depth>=self.max_depth):
            if x_train["Survived"].mean()>=0.5:
                self.target=1
            else:
                self.target=0
            return
        
        self.left=decision_tree(self.max_depth,self.depth+1)
        self.left.train(left)
        self.right=decision_tree(self.max_depth,self.depth+1)
        self.right.train(right)
        return
        
            
    def predict(self,test):
        if test[self.fkey]>self.fval:
            if self.right is None:
                return int(self.target)
            return self.right.predict(test)
            

        else:
            if self.left is None:
                return int(self.target)
            return self.left.predict(test)
            
                             
        
        
        
        
        

In [632]:
x_train=frame[:int(len(frame)*0.8)]
x_test=frame[int(len(frame)*0.8):]

In [633]:
tree=decision_tree()
tree.train(x_train)

flag_sex
Pclass
Age
SibSp


C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log2
  after removing the cwd from sys.path.
C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


Pclass
Age
Age
Age
Age
Age
Age
Pclass
Parch
Age
Age
Fare
Pclass
Age
Parch
Fare
Age
Age
Age
Age
Pclass
Age
Age
Age
Age
Age
Pclass
Parch
Age
Fare
Age
SibSp
Parch
Pclass
Age
Age
Fare
Age
Age
Age
Pclass
Age
Pclass
Age
Age
Age
Age
Pclass
Age
Pclass
Age
Pclass
Pclass
Age
Pclass
Parch
Pclass
Pclass
SibSp
Parch
Fare
Pclass
Age
Age
Pclass
Age
Age
Age
Age
Age
Age
Age
Age
Age
Pclass
Pclass
Age
Age
Pclass
Pclass
Age
Age
Pclass
Age
Age
Pclass
Pclass
Age
Pclass
Pclass
Pclass
Age
Age
Age
Age
Fare
Age
Age
Pclass
Age
Age
Age
Age
Age
Pclass
Pclass
Age
Age
Age
Pclass
Pclass
Pclass
Age
Age
Pclass
Pclass
Pclass
Parch
Age
Fare
Fare
SibSp
Age
Fare
Fare
Pclass
Age
Age
Pclass
SibSp
Age
Age
Pclass
SibSp
SibSp
Pclass
Age
Age
Age
Pclass
Age
Fare
Age
Age
Pclass
Fare
Pclass
Age
SibSp
Fare
Pclass
Pclass
Fare
Age
Age
Pclass
Age
Age
Pclass
Pclass
Age
Age
Age
Pclass
Pclass
Fare
Age
Parch
Age
Age
Age
Age
Age
Age
Pclass
Age
Age
Pclass
Pclass
Pclass
Fare
Age
SibSp
Age
Age
Age
Pclass
Pclass
SibSp
Age
Age
Pclass
Age
Age
Pcl

In [634]:
x_test.reset_index(inplace=True,drop=True)
y_test=[]
for i in range(len(x_test)):
    y_test.append(tree.predict(x_test.iloc[i]))

In [635]:
print("accuracy is ",(np.sum(np.array(y_test)==x_test.Survived)/len(x_test))*100)


accuracy is  87.70949720670392
